# Accuinsight+ Modeler AutoDL tutorial
## 스팸 계정과 정상 계정을 분류하는 MLP 모델  
## 목차
1. [분석 개요](#mlp_분석개요)
2. [데이터](#mlp_데이터)
3. [MLP 모델 생성 및 학습](#mlp_모델_구성_및_학습)
4. [Modeler AutoDL 사용방법](#AutoDL_사용방법)

<a id='mlp_분석개요'></a>
## 분석 개요
- 본 예제에서는 kaggle에서 제공하는 [인스타그램 데이터](https://www.kaggle.com/free4ever1/instagram-fake-spammer-genuine-accounts?select=train.csv)를 사용해 스팸 계정과 정상 계정을 분류하는 MLP 모델을 만들고,  
  Accuinsight+ Modeler의 AutoDL 기능을 사용하여 하이퍼파라미터를 최적화하는 과정을 설명합니다.([이미지 출처](https://www.intechopen.com/books/artificial-neural-networks-architectures-and-applications/mlp-and-anfis-applied-to-the-prediction-of-hole-diameters-in-the-drilling-process/))
<p align="center">
    <img src="pic/MLP.jpg" height="400px" width="500px">
</p>

In [6]:
import tensorflow as tf
import numpy as np
import pandas as pd
import easydict
import csv
from datetime import datetime, timezone

from tensorflow.keras.callbacks import Callback
from tensorflow.keras.layers import Dense
from tensorflow.keras import optimizers, Input

<a id='mlp_데이터'></a>
## 데이터
- 설명 변수는 총 11개로, 4개의 이진 변수를 포함하고 있습니다. 
  - 이진 변수: `profile pic`, `name==username`, `external URL`, `private`
- 데이터의 클래스를 의미하는 변수는 `fake`이고 `fake`의 값이 1이면 스팸 계정이고 0이면 정상 계정입니다.

train 데이터를 불러옵니다.

In [4]:
train = pd.read_csv("data/instagram_train.csv")
train.head()

,profile pic,nums/length username,fullname words,nums/length fullname,name==username,description length,external URL,private,#posts,#followers,#follows,fake
0,1,0.27,0,0.0,0,53,0,0,32,1000,955,0
1,1,0.00,2,0.0,0,44,0,0,286,2740,533,0
2,1,0.10,2,0.0,0,0,0,1,13,159,98,0
3,1,0.00,1,0.0,0,82,0,0,679,414,651,0
4,1,0.00,2,0.0,0,0,0,1,6,151,126,0


In [3]:
train['profile pic'] = train['profile pic'].astype('category')
train['name==username'] = train['name==username'].astype('category')
train['external URL'] = train['external URL'].astype('category')
train['private'] = train['private'].astype('category')

train_X = pd.get_dummies(train)
train_Y = train_X.pop("fake")

test 데이터를 불러옵니다.

In [4]:
test = pd.read_csv("data/instagram_test.csv")
test.head()

,profile pic,nums/length username,fullname words,nums/length fullname,name==username,description length,external URL,private,#posts,#followers,#follows,fake
0,1,0.33,1,0.33,1,30,0,1,35,488,604,0
1,1,0.00,5,0.00,0,64,0,1,3,35,6,0
2,1,0.00,2,0.00,0,82,0,1,319,328,668,0
3,1,0.00,1,0.00,0,143,0,1,273,14890,7369,0
4,1,0.50,1,0.00,0,76,0,1,6,225,356,0


In [5]:
test['profile pic'] = test['profile pic'].astype('category')
test['name==username'] = test['name==username'].astype('category')
test['external URL'] = test['external URL'].astype('category')
test['private'] = test['private'].astype('category')

test_X = pd.get_dummies(test)
test_Y = test_X.pop("fake")

데이터 개수를 확인합니다.

In [6]:
print("총 데이터 개수: ", len(train) + len(test))
print("train 데이터 개수: ", len(train))
print("test 데이터 개수: ", len(test))

총 데이터 개수:  696
train 데이터 개수:  576
test 데이터 개수:  120


<a id='mlp_모델_구성_및_학습'></a>
## MLP 모델 생성 및 학습

- 본 예제에서는 모델 학습과 관련된 하이퍼파라미터인 epochs, batch size, learning_rate와  
  모델 구조를 결정하는 하이퍼파라미터인 num_nodes와 num_layers를 최적화합니다.  
  - `num_nodes`: Hidden layer의 노드 개수
  - `num_layers`: Hidden layer의 개수
- AutoDL을 사용하기 위해서는 __argparse__를 통해 하이퍼파라미터를 설정해야 하는데,  
  jupyter notebook에서는 argparse 사용이 불가능하기 때문에 __easydict__를 통해 하이퍼파라미터를 설정합니다.  
  실제로 AutoDL에 사용되는 [autodl_mlp_user_classification.py](autodl_mlp_user_classification.py)에서는 argparse를 통해 하이퍼파라미터를 설정합니다.

In [3]:
args = easydict.EasyDict({
    "epochs": 100,
    "batch_size": 64,
    "learning_rate": 0.0001,
    "activation_func": 'relu',
    "num_nodes": 16,
    "num_layers": 2})

MLP 모델을 생성합니다.

In [4]:
model = tf.keras.Sequential()
model.add(Input(shape=(15,)))
for i in range(args.num_layers):
    model.add(Dense(args.num_nodes, activation=args.activation_func))
model.add(Dense(2, activation=args.activation_func))

# Adam optimizer를 사용합니다.
opt = optimizers.Adam(lr=args.learning_rate, amsgrad=True)
model.compile(optimizer=opt,
              loss='binary_crossentropy',
              metrics = ['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 16)                256       
_________________________________________________________________
dense_1 (Dense)              (None, 16)                272       
_________________________________________________________________
dense_2 (Dense)              (None, 2)                 34        
Total params: 562
Trainable params: 562
Non-trainable params: 0
_________________________________________________________________


- AutoDL에서 평가 지표를 수집할 수 있도록 epoch이 종료될 때마다 평가 지표(Accuracy)를 출력합니다.
- 이번 예제에서는 __평가 지표명=평가 지표값__ 형식으로 평가 지표를 출력하는데,  
  평가 지표 설정 방법은 [Modeler AutoDL 사용방법](#AutoDL_사용방법) 챕터에서 좀 더 자세히 다룰 것입니다.

In [18]:
class MetricHistory(Callback):
    def on_epoch_end(self, epoch, logs=None):
        local_time = datetime.now(timezone.utc).astimezone().isoformat()
        print("\nEpoch {}".format(epoch + 1))
        print("{} Train-acc={:.4f}".format(local_time, logs['accuracy']))
        print("{} Validation-acc={:.4f}".format(local_time, logs['val_accuracy']))

history = MetricHistory()

model.fit(train_X.values, train_Y.values,
          epochs=args.epochs,
          batch_size=args.batch_size,
          validation_data=(test_X, test_Y),
          callbacks=[history])

Epoch 1/100
1/9 [==>...........................] - ETA: 0s - loss: 6.6845 - accuracy: 0.5938
Epoch 1
2020-09-21T23:14:48.250296+09:00 Train-acc=0.5781
2020-09-21T23:14:48.250296+09:00 Validation-acc=0.6167
9/9 [==============================] - 0s 14ms/step - loss: 6.8033 - accuracy: 0.5781 - val_loss: 6.8933 - val_accuracy: 0.6167
Epoch 2/100
1/9 [==>...........................] - ETA: 0s - loss: 7.0154 - accuracy: 0.6094
Epoch 2
2020-09-21T23:14:48.304365+09:00 Train-acc=0.5816
2020-09-21T23:14:48.304365+09:00 Validation-acc=0.6167
9/9 [==============================] - 0s 5ms/step - loss: 6.7881 - accuracy: 0.5816 - val_loss: 6.8415 - val_accuracy: 0.6167
Epoch 3/100
1/9 [==>...........................] - ETA: 0s - loss: 6.4006 - accuracy: 0.5938
Epoch 3
2020-09-21T23:14:48.454593+09:00 Train-acc=0.5868
2020-09-21T23:14:48.454593+09:00 Validation-acc=0.6167
9/9 [==============================] - 0s 16ms/step - loss: 6.7717 - accuracy: 0.5868 - val_loss: 6.8878 - val_accuracy: 0.6167

<a id='AutoDL_사용방법'></a>
## Modeler AutoDL 사용방법
- 지금까지 스팸 계정과 정상 계정을 분류하는 MLP 모델 예제를 살펴보았습니다.
- 이번 챕터에서는 위에서 살펴본 MLP 모델의 하이퍼파라미터를 AutoDL 기능을 사용해 최적화하는 과정에 대해 다룰 것입니다.